In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
"""
Calcula a probabilidade de um resultado possível da distribuição multinominal
para um total de 4 escolhas possíveis
"""


def p(i, j, k, l):
    n = i + j + k + l
    output = (
        math.factorial(n)
        / (
            math.factorial(i)
            * math.factorial(j)
            * math.factorial(k)
            * math.factorial(l)
        )
        * (1 / 4) ** n
    )

    return output

In [ ]:
n = 100
result = 0
for i in range(n + 1):
    for j in range(n + 1 - i):
        for k in range(n + 1 - i - j):
            l = n - i - j - k

            result += p(i, j, k, l)

print(result)

In [ ]:
n = 100
result = np.array([0.0, 0.0])
for i in range(n + 1):
    for j in range(n + 1 - i):
        for k in range(n + 1 - i - j):
            l = n - i - j - k

            result += p(i, j, k, l) * (
                i * np.array([1, 0])
                + j * np.array([0, 1])
                + k * np.array([-1, 0])
                + l * np.array([0, -1])
            )

print(result)

In [ ]:
# Variância
n = 100
result = np.array([0.0, 0.0])
for i in range(n + 1):
    for j in range(n + 1 - i):
        for k in range(n + 1 - i - j):
            l = n - i - j - k

            result += (
                p(i, j, k, l)
                * (
                    i * np.array([1, 0])
                    + j * np.array([0, 1])
                    + k * np.array([-1, 0])
                    + l * np.array([0, -1])
                )
                ** 2
            )

print(result)

In [ ]:
# Distância média à origem
result = 0.0
for i in range(n + 1):
    for j in range(n + 1 - i):
        for k in range(n + 1 - i - j):
            l = n - i - j - k

            result += p(i, j, k, l) * np.sqrt(
                np.sum(
                    (
                        i * np.array([1, 0])
                        + j * np.array([0, 1])
                        + k * np.array([-1, 0])
                        + l * np.array([0, -1])
                    )
                    * (
                        i * np.array([1, 0])
                        + j * np.array([0, 1])
                        + k * np.array([-1, 0])
                        + l * np.array([0, -1])
                    )
                )
            )

print(result)

In [ ]:
def inicia(nPassos, nAmostras):
    
    '''Faz a inicialização das estruturas de dados que vão armazenar os resultados
    das várias amostragens de caminhos aleatórios
    nPassos: O número de passos aleatórios do caminho
    nAmostras: O número de amostragens a efectuar
    return: Tupel com os diferentes arrays'''
    
    tipo = 'float'
    
    sx = np.zeros(nPassos + 1, dtype = tipo) #A soma das posições em x no passo i
    sy = np.zeros(nPassos + 1, dtype = tipo) #A soma das posições em y no passo i
    sr = np.zeros((nPassos + 1, 2), dtype = tipo) #A soma dos vectores posição no passo i
    sx2 = np.zeros(nPassos + 1, dtype = tipo) #A soma das posições em x ao quadrado no passo i
    sy2 = np.zeros(nPassos + 1, dtype = tipo) #A soma das posições em y ao quadrado no passo i
    sr2 = np.zeros(nPassos + 1, dtype = tipo) #A soma dos vectores posição ao quadrado no passo i
    sNorma = np.zeros(nPassos + 1, dtype = tipo) #A soma das distâncias à origem no passo i
    rf = np.zeros((nAmostras, 2), dtype = tipo) #A posição final em cada amostagem
    
    return sx, sy, sr, sx2, sy2, sr2, sNorma, rf

In [ ]:
def passoSimples():
    
    '''Produz um passo num caminho aleatório simples e sem direcção preferencial'''
    
    direccao = np.random.randint(4)
    possiveis = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]], dtype = 'int')
    
    return possiveis[direccao]

In [ ]:
def fazCaminho(nPassos, data):
    '''Faz um caminho aleatório registando os valores em cada passo na estrutura de dados
    nPassos: Número de passos do caminho
    data: Estrutura de dados para armazenar os valores
    return: O caminho aleatório'''
    
    caminho = np.zeros((nPassos + 1, 2), dtype = 'int') #Inicializa o caminho
    #Cria o caminho aleatório e guarda os dados em cada passo
    for i in range(1, nPassos + 1):
        
        passo = passoSimples()
        
        caminho[i] = caminho[i - 1] + passo
        
        data[0][i] += caminho[i, 0] #sx para cada passo 0..100 a soma do valor em x das nAmostragens
        data[1][i] += caminho[i, 1] #sy
        data[2][i] += caminho[i] #sr
        data[3][i] += caminho[i, 0] ** 2 #sx2
        data[4][i] += caminho[i, 1] ** 2 #sy2
        data[5][i] += np.sum(caminho[i] * caminho[i]) #sr2
        data[6][i] += np.sum(caminho[i] * caminho[i]) ** (1/2) #sNorma
    
    return caminho

In [ ]:
def amostragemCaminhos(nPassos, nAmostragens, grafico = True):
    '''Cria nAmostragens de caminhos aleatórios com nPassos
    nPassos: Número de passos nos caminhos
    nAmostragens: Número de caminhos a produzir
    grafico: Se True faz o gráfico do último caminho.
    return: Os dados cumulativos das várias amostragens por passo e posições finais'''
    
    data = inicia(nPassos, nAmostragens) #Inicializa as estruturas de dados para guardar resultados
    
    for i in range(nAmostragens):
        
        caminho = fazCaminho(nPassos, data) #Produz um caminho
        
        data[7][i] = caminho[-1] #Guarda a última posição do caminho
    
    data[0][:] = data[0] / nAmostragens
    data[1][:] = data[1] / nAmostragens
    data[2][:] = data[2] / nAmostragens
    data[3][:] = data[3] / nAmostragens
    data[4][:] = data[4] / nAmostragens
    data[5][:] = data[5] / nAmostragens
    data[6][:] = data[6] / nAmostragens
    
    if grafico:
        fig, ax = plt.subplots(figsize = (8, 8))
        ax.plot(caminho[:, 0], caminho[:, 1])
        
    return data

In [ ]:
out = amostragemCaminhos(100, 10000)

In [ ]:
figRF, axRF = plt.subplots(figsize = (8,8))
axRF.plot(out[7][:, 0], out[7][:, 1], 'o')
axRF.set_xlabel('x')
axRF.set_ylabel('y')
axRF.set_title('Posições finais')
pass

In [ ]:
#X médio
figX, axX = plt.subplots(figsize = (8,8))
axX.plot(out[0], 'o')
axX.set_title('$\overline{x}$ vs #passo')
axX.set_ylabel(r'$\overline{x}$')
axX.set_xlabel(r'# passo')
pass

In [ ]:
#Y médio
figY, axY = plt.subplots(figsize = (8,8))
axY.plot(out[1], 'o')
axY.set_title('$\overline{y}$ vs #passo')
axY.set_ylabel(r'$\overline{y}')
axY.set_xlabel(r'# passo')
pass

In [ ]:
#SX2 médio
figSX2, axSX2 = plt.subplots(figsize = (8,8))
axSX2.plot(out[3], 'o')
axSX2.set_title('$\sigma_x^2$ vs #passo')
axSX2.set_ylabel(r'$\sigma_x^2$')
axSX2.set_xlabel(r'# passo')
pass

In [ ]:
#SY2 médio
figSY2, axSY2 = plt.subplots(figsize = (8,8))
axSY2.plot(out[4], 'o')
axSY2.set_title('$\sigma_y^2$ vs #passo')
axSY2.set_ylabel(r'$\sigma_y^2$')
axSY2.set_xlabel(r'# passo')
pass

In [ ]:
#SR2 médio
figSR2, axSR2 = plt.subplots(figsize = (8,8))
axSR2.plot(out[5], 'o')
axSR2.set_title('$\sigma_r^2$ vs #passo')
axSR2.set_ylabel(r'$\sigma_r^2$')
axSR2.set_xlabel(r'# passo')
pass

In [ ]:
#SRN médio
figRN, axRN = plt.subplots(figsize = (8,8))
axRN.plot(out[6], 'o')
axRN.set_title('$\overline{r}$ vs #passo')
axRN.set_ylabel(r'$\overline{r}$')
axRN.set_xlabel(r'# passo')
pass


In [ ]:
norm = np.sum((out[7] * out[7]), axis = 1) ** (1/2)
figH, axH = plt.subplots(figsize = (8, 8))
axH.hist(norm, bins = 15)
pass

# Caminho aleatório com direção preferencial

Os caminhos aleatórios podem ter uma ou mais direcções preferências, em que a probabilidade do passo ser numa dada direcção pode ser diferente das outras direcções. Neste caso ainda é possível obter facilmente de forma analítica os valores médios e variâncias esperados, bastando alterar as probabilidades de cada direcção para outros valores. Deixa-se como exercício para os alunos obterem estes valores esperados.

Podemos manter maior parte das funções usadas anteriormente, apenas substituindo a função passoSimples e pequenas alterações nas restantes funções.

In [ ]:
def passoPreferencial(pi, pj, pk, pl):
    '''Calcula um passo de um caminho aleatório em que as direcções +x, +y, -x e -y tem os
    pesos dados pelos argumentos da função.
    pi: Peso da direcção +x
    pj: Peso da direcção +y
    pk: Peso da direcção -x
    pl: Peso da direcção -y
    
    return: O vector com a direcção do passo'''
    
    total = pi + pj + pk + pl #Número total dos pesos
    
    possiveis = np.array(pi * [[1, 0]] + pj * [[0, 1]] + pk * [[-1, 0]] + pl * [[0, -1]]) #Constroi o array com todas as hpóteses
    
    direccao = np.random.randint(total)
    
    return possiveis[direccao]

In [ ]:
def fazCaminho(nPassos, data, tipo = None):
    '''Faz um caminho aleatório registando os valores em cada passo na estrutura de dados
    nPassos: Número de passos do caminho
    data: Estrutura de dados para armazenar os valores
    tipo: None para caminho simples, tuple com os pesos de cada direcção
    return: O caminho aleatório'''
    
    caminho = np.zeros((nPassos + 1, 2), dtype = 'int') #Inicializa o caminho
    #Cria o caminho aleatório e guarda os dados em cada passo
    for i in range(1, nPassos + 1):
        
        if tipo is None:
            passo = passoSimples()
        else:
            passo = passoPreferencial(tipo[0], tipo[1], tipo[2], tipo[3])
        
        caminho[i] = caminho[i - 1] + passo
        
        data[0][i] += caminho[i, 0] #sx
        data[1][i] += caminho[i, 1] #sy
        data[2][i] += caminho[i] #sr
        data[3][i] += caminho[i, 0] ** 2 #sx2
        data[4][i] += caminho[i, 1] ** 2 #sy2
        data[5][i] += np.sum(caminho[i] * caminho[i]) #sr2
        data[6][i] += np.sum((caminho[i] * caminho[i])) ** (1/2) #sNorma
    
    return caminho

In [ ]:
def amostragemCaminhos(nPassos, nAmostragens, grafico = True, tipo = None):
    '''Cria nAmostragens de caminhos aleatórios com nPassos
    nPassos: Número de passos nos caminhos
    nAmostragens: Número de caminhos a produzir
    grafico: Se True faz o gráfico do último caminho.
    tipo: Direcções preferenciais, None para não haver direcções preferencias, tuple com os pesos de cada direcção
    return: Os dados cumulativos das várias amostragens por passo e posições finais'''
    
    data = inicia(nPassos, nAmostragens) #Inicializa as estruturas de dados para guardar resultados
    
    for i in range(nAmostragens):
        
        caminho = fazCaminho(nPassos, data, tipo) #Produz um caminho
        
        data[7][i] = caminho[-1] #Guarda a última posição do caminho
    
    data[0][:] = data[0] / nAmostragens
    data[1][:] = data[1] / nAmostragens
    data[2][:] = data[2] / nAmostragens
    data[3][:] = data[3] / nAmostragens
    data[4][:] = data[4] / nAmostragens
    data[5][:] = data[5] / nAmostragens
    data[6][:] = data[6] / nAmostragens
    
    if grafico:
        fig, ax = plt.subplots(figsize = (8, 8))
        ax.plot(caminho[:, 0], caminho[:, 1])
        
    return data

In [ ]:
def xyGraficos(data):
    '''Produz os gráficos para análise de caminhos aleatórios, x e y médio, 
    variâncias de x e y
    data: Array com a informação de x, y, r, x**2, y**2, r**2, norma médios e posições finais'''
    
    fig = plt.Figure()
    axX = fig.add_subplot(2, 2, 1)
    axX.plot(data[0], 'o')
    axX.set_title('$\overline{x}$ vs #passo')
    axX.set_ylabel(r'$\overline{x}$')
    axX.set_xlabel(r'# passo')
    
    axY = fig.add_subplot(2, 2, 2)
    axY.plot(data[1], 'o')
    axY.set_title('$\overline{y}$ vs #passo')
    axY.set_ylabel(r'$\overline{y}$')
    axY.set_xlabel(r'# passo')
    
    axSX2 = fig.add_subplot(2, 2, 3)
    axSX2.plot(data[3] - data[0] ** 2, 'o')
    axSX2.set_title('$\sigma_x^2$ vs #passo')
    axSX2.set_ylabel(r'$\sigma_x^2$')
    axSX2.set_xlabel(r'# passo')
    
    axSY2 = fig.add_subplot(2, 2, 4)
    axSY2.plot(data[4] - data[1] ** 2, 'o')
    axSY2.set_title('$\sigma_y^2$ vs #passo')
    axSY2.set_ylabel(r'$\sigma_y^2$')
    axSY2.set_xlabel(r'# passo')
    
    fig.set_size_inches(12, 12)
    
    return fig

In [ ]:
def distanciaGraficos(data):
    
    '''Produz os gráficos para análise de caminhos aleatórios, distância à origem, histograma e posições finais
    data: Array com a informação de x, y, r, x**2, y**2, r**2, norma médios e posições finais'''
    
    fig = plt.Figure()
    
    axSR2 = fig.add_subplot(2, 2, 1)
    axSR2.plot(data[5] - (data[0] ** 2 + data[1] ** 2), 'o')
    axSR2.set_title('$\sigma_r^2$ vs #passo')
    axSR2.set_ylabel(r'$\sigma_r^2$')
    axSR2.set_xlabel(r'# passo')
    
    axRN = fig.add_subplot(2, 2, 2)
    axRN.plot(data[6], 'o')
    axRN.set_title('$\overline{r}$ vs #passo')
    axRN.set_ylabel(r'$\overline{r}$')
    axRN.set_xlabel(r'# passo')
    
    norm = np.sum((data[7] * data[7]), axis = 1) ** (1/2)
    axH = fig.add_subplot(2, 2, 3)
    axH.hist(norm, bins = 15)
    
    axRF = fig.add_subplot(2, 2, 4)
    axRF.plot(data[7][:, 0], data[7][:, 1], 'o')
    axRF.set_xlabel('x')
    axRF.set_ylabel('y')
    axRF.set_title('Posições finais')
    
    fig.set_size_inches(12, 12)
    
    return fig

In [ ]:
data = amostragemCaminhos(100, 10000, tipo = (4, 1, 1, 1))

In [ ]:
fig1 = xyGraficos(data)
fig1

In [ ]:
fig2 = distanciaGraficos(data)
fig2

# Caminho aleatório que se auto evita

Um caminho aleatório que já não cumpre as definições de caminho aleatório simples e que já não permite uma análise analítica dos resultados esperados é o caminho aleatório que se auto evita.

Este caminho termina quando volta a uma posição que já tinha estado anteriormente.

Este caminho cria mais uma variável de interesse que é quantos passos cada caminho na amostragem deu antes de terminar.

Vamos assim alterar mais uma vez algumas das funções já usadas para produzir e analisar estes caminhos aleatórios.

In [ ]:
def inicia(nPassos, nAmostras):
    
    '''Faz a inicialização das estruturas de dados que vão armazenar os resultados
    das várias amostragens de caminhos aleatórios
    nPassos: O número de passos aleatórios do caminho
    nAmostras: O número de amostragens a efectuar
    return: Tupel com os diferentes arrays'''
    
    tipo = 'float'
    
    sx = np.zeros(nPassos + 1, dtype = tipo) #A soma das posições em x no passo i
    sy = np.zeros(nPassos + 1, dtype = tipo) #A soma das posições em y no passo i
    sr = np.zeros((nPassos + 1, 2), dtype = tipo) #A soma dos vectores posição no passo i
    sx2 = np.zeros(nPassos + 1, dtype = tipo) #A soma das posições em x ao quadrado no passo i
    sy2 = np.zeros(nPassos + 1, dtype = tipo) #A soma das posições em y ao quadrado no passo i
    sr2 = np.zeros(nPassos + 1, dtype = tipo) #A soma dos vectores posição ao quadrado no passo i
    sNorma = np.zeros(nPassos + 1, dtype = tipo) #A soma das distâncias à origem no passo i
    rf = np.zeros((nAmostras, 2), dtype = tipo) #A posição final em cada amostagem
    tamanho = np.zeros(nAmostras, dtype = tipo) #O número de passos realizados antes do caminho terminar
    dim = np.zeros(nPassos + 1, dtype = tipo) #O número de caminhos que chegou ao passo i, para dividir os arrays ligados
    #aos passos e obter médias
    
    return sx, sy, sr, sx2, sy2, sr2, sNorma, rf, tamanho, dim

In [ ]:
def passoSimples():
    
    '''Produz um passo num caminho aleatório simples e sem direcção preferencial'''
    
    direccao = np.random.randint(4)
    possiveis = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]], dtype = 'int')
    
    return possiveis[direccao]

In [ ]:
def fazCaminhoEvita(nPassos, data):
    '''Faz um caminho aleatório registando os valores em cada passo na estrutura de dados
    nPassos: Número de passos do caminho
    data: Estrutura de dados para armazenar os valores
    return: O caminho aleatório'''
    caminho = np.zeros((nPassos + 1, 2), dtype = 'int') #Inicializa o caminho
    data[9][0] += 1 #A posição inicial é efectuada todos os caminhos
    #Cria o caminho aleatório e guarda os dados em cada passo
    for i in range(1, nPassos + 1):
        
        passo = passoSimples()
        posicaoTmp = caminho[i - 1] + passo
        visitado = False
        for j in range(0, i + 1):
            visitado = caminho[-nPassos + j, 0] == posicaoTmp[0] and caminho[-nPassos + j, 1] == posicaoTmp[1]
            if visitado:
                break
        if visitado:
            break
        
        caminho[i] = posicaoTmp
        
        data[0][i] += caminho[i, 0] #sx
        data[1][i] += caminho[i, 1] #sy
        data[2][i] += caminho[i] #sr
        data[3][i] += caminho[i, 0] ** 2 #sx2
        data[4][i] += caminho[i, 1] ** 2 #sy2
        data[5][i] += np.sum(caminho[i] * caminho[i]) #sr2
        data[6][i] += np.sum((caminho[i] * caminho[i])) ** (1/2) #sNorma
        data[9][i] += 1
    
    return caminho, i

In [ ]:
def amostragemCaminhos(nPassos, nAmostragens, grafico = True):
    '''Cria nAmostragens de caminhos aleatórios com nPassos
    nPassos: Número de passos nos caminhos
    nAmostragens: Número de caminhos a produzir
    grafico: Se True faz o gráfico do último caminho.
    return: Os dados cumulativos das várias amostragens por passo e posições finais'''

    data = inicia(nPassos, nAmostragens) #Inicializa as estruturas de dados para guardar resultados
    
    for i in range(nAmostragens):
        
        caminho, n = fazCaminhoEvita(nPassos, data) #Produz um caminho
        
        data[7][i] = caminho[n - 1] #Guarda a última posição do caminho
        data[8][i] = n #Guarda o número de passos do caminho efectuado
    
    maxN = int(data[8].max())
    
    data[0][:maxN] = data[0][:maxN] / data[9][ :maxN]
    data[1][:maxN] = data[1][:maxN] / data[9][ :maxN]
    #data[2][:] = data[2] / data[9]
    data[3][:maxN] = data[3][:maxN] / data[9][ :maxN]
    data[4][:maxN] = data[4][:maxN] / data[9][ :maxN]
    data[5][:maxN] = data[5][:maxN] / data[9][ :maxN]
    data[6][:maxN] = data[6][:maxN] / data[9][ :maxN]
    
    if grafico:
        fig, ax = plt.subplots(figsize = (8, 8))
        ax.plot(caminho[:n, 0], caminho[:n, 1])
        
    return data

In [ ]:
data = amostragemCaminhos(100, 10000)

In [ ]:
fig1 = xyGraficos(data)
fig1

In [ ]:
fig2 = distanciaGraficos(data)
fig2

In [ ]:
data[9]

In [ ]:
def passosGraficos(data):
    
    '''Produz os gráficos para análise de caminhos aleatórios, passos efectuados
    data: Array com a informação de x, y, r, x**2, y**2, r**2, norma médios e posições finais'''
    
    fig = plt.Figure()
    
    ax = fig.add_subplot(221)
    ax.plot(data[8])
    ax.set_xlabel('# amostra')
    ax.set_ylabel('passos')
    ax.set_title('# passos vs amostragem')
    
    ax = fig.add_subplot(222)
    ax.plot(data[9], 'o')
    ax.set_xlabel('passo')
    ax.set_ylabel('amostragens')
    ax.set_title('amostragem vs passo')
    
    ax = fig.add_subplot(223)
    ax.hist(data[8], bins = 15)
    
    fig.set_size_inches(12, 12)
    return fig

In [ ]:
fig3 = passosGraficos(data)
fig3

In [ ]:
def passosEstatistica(data):
    
    media = data[8].mean()
    desvio = data[8].std()
    maximo = data[8].max()
    mediana = np.median(data[8])
    
    print('Média:\t', media)
    print('Desvio padrão:\t', desvio)
    print('Mediana:\t', mediana)
    print('Máximo:\t', maximo)

In [ ]:
passosEstatistica(data)

In [ ]:
data = inicia(100, 100)

In [ ]:
%timeit -n 10000 -r 10 fazCaminhoEvita(100, data)

# Passo perpendiculares

Podemos aumentar a longevidade dos caminhos aleatórios que se auto evitam obrigando cada passo a ser perpendicular ao anterior.

In [ ]:
def passoPerpendicular(passoAnterior = 0):
    
    '''Produz um passo num caminho aleatório perpendicular ao passo anterior
    passoAnterior: Direcção do passo anterior, 1 se o anterior foi horizontal,
    -1 se foi vertical e 0 se é o primeiro passo'''
    
    if passoAnterior > 0:
        direccao = np.random.randint(2)
        possiveis = np.array([[0, 1], [0, -1]], dtype = 'int')
        passoAnterior = -1
    elif passoAnterior < 0:
        direccao = np.random.randint(2)
        possiveis = np.array([[1, 0], [-1, 0]], dtype = 'int')
        passoAnterior = 1
    else:
        direccao = np.random.randint(4)
        possiveis = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]], dtype = 'int') 
        passoAnterior = 1 if (direccao % 2) == 0 else -1
    
    return possiveis[direccao], passoAnterior

In [ ]:
def fazCaminhoEvitaPrep(nPassos, data):
    '''Faz um caminho aleatório registando os valores em cada passo na estrutura de dados
    nPassos: Número de passos do caminho
    data: Estrutura de dados para armazenar os valores
    return: O caminho aleatório'''
    caminho = np.zeros((nPassos + 1, 2), dtype = 'int') #Inicializa o caminho
    data[9][0] += 1 #A posição inicial é efectuada todos os caminhos
    #Cria o caminho aleatório e guarda os dados em cada passo
    direccao = 0
    for i in range(1, nPassos + 1):
        
        passo, direccao = passoPerpendicular(direccao)
        posicaoTmp = caminho[i - 1] + passo
        visitado = False
        for j in range(0, i + 1):
            visitado = caminho[-nPassos + j, 0] == posicaoTmp[0] and caminho[-nPassos + j, 1] == posicaoTmp[1]
            if visitado:
                break
        if visitado:
            break
        
        caminho[i] = posicaoTmp
        
        data[0][i] += caminho[i, 0] #sx
        data[1][i] += caminho[i, 1] #sy
        data[2][i] += caminho[i] #sr
        data[3][i] += caminho[i, 0] ** 2 #sx2
        data[4][i] += caminho[i, 1] ** 2 #sy2
        data[5][i] += np.sum(caminho[i] * caminho[i]) #sr2
        data[6][i] += np.sum((caminho[i] * caminho[i])) ** (1/2) #sNorma
        data[9] [i] += 1
    else:
        i += 1
    
    return caminho, i

In [ ]:
def amostragemCaminhosPrep(nPassos, nAmostragens, grafico = True):
    '''Cria nAmostragens de caminhos aleatórios com nPassos
    nPassos: Número de passos nos caminhos
    nAmostragens: Número de caminhos a produzir
    grafico: Se True faz o gráfico do último caminho.
    return: Os dados cumulativos das várias amostragens por passo e posições finais'''
    
    data = inicia(nPassos, nAmostragens) #Inicializa as estruturas de dados para guardar resultados
    
    for i in range(nAmostragens):
        
        caminho, n = fazCaminhoEvitaPrep(nPassos, data) #Produz um caminho
        
        data[7][i] = caminho[n - 1] #Guarda a última posição do caminho
        data[8][i] = n #Guarda o número de passos do caminho efectuado
    
    maxN = int(data[8].max())
    
    data[0][:maxN] = data[0][:maxN] / data[9][ :maxN]
    data[1][:maxN] = data[1][:maxN] / data[9][ :maxN]
    #data[2][:] = data[2] / data[9]
    data[3][:maxN] = data[3][:maxN] / data[9][ :maxN]
    data[4][:maxN] = data[4][:maxN] / data[9][ :maxN]
    data[5][:maxN] = data[5][:maxN] / data[9][ :maxN]
    data[6][:maxN] = data[6][:maxN] / data[9][ :maxN]
    
    if grafico:
        fig, ax = plt.subplots(figsize = (8, 8))
        ax.plot(caminho[:n, 0], caminho[:n, 1])
        
    return data

In [ ]:
data = amostragemCaminhosPrep(100, 10000)

In [ ]:
fig1 = xyGraficos(data)
fig1

In [ ]:
fig2 = distanciaGraficos(data)
fig2

In [ ]:
fig3 = passosGraficos(data)
fig3

In [ ]:
passosEstatistica(data)

In [ ]:
data = inicia(100, 100)

In [ ]:
%timeit -n 10000 -r 10 fazCaminhoEvitaPrep(100, data)

# Caminho aleatório que se auto evita crescente

Com o objectivo de aumentar o número de passos efectuados no caminho aleatório podemos ainda permitir que sejam efectuadas várias tentativas de obter um passo que não leve a um ponto por onde o caminho já tenha passado.

In [ ]:
def fazCaminhoEvitaPrepCres(nPassos, data):
    '''Faz um caminho aleatório registando os valores em cada passo na estrutura de dados
    nPassos: Número de passos do caminho
    data: Estrutura de dados para armazenar os valores
    return: O caminho aleatório'''
    caminho = np.zeros((nPassos + 1, 2), dtype = 'int') #Inicializa o caminho
    data[9][0] += 1 #A posição inicial é efectuada todos os caminhos
    #Cria o caminho aleatório e guarda os dados em cada passo
    direccao = 0 #Direccção do último passo, 0 primeiro passo, 1 horizontal, -1 vertical
    maxTentativas = 20 #N máximo de tentativas para encontrar um passo que não leve a uma posição já visitada
    for i in range(1, nPassos + 1):
        
        tentativas = 0 #n de tentativas efectuadas
        while tentativas < maxTentativas:
            tentativas += 1
            passo, direccaoTmp = passoPerpendicular(direccao)
            posicaoTmp = caminho[i - 1] + passo
            visitado = False
            for j in range(0, i + 1):
                visitado = caminho[-nPassos + j, 0] == posicaoTmp[0] and caminho[-nPassos + j, 1] == posicaoTmp[1]
                if visitado:
                    break
            if not visitado: #Se a nova posição não tiver sido visitada sair do while
                break
        else:
            break #Se o while tiver terminado por exceder o n de tentativas terminar o caminho

        caminho[i] = posicaoTmp
        direccao = direccaoTmp
        
        data[0][i] += caminho[i, 0] #sx
        data[1][i] += caminho[i, 1] #sy
        data[2][i] += caminho[i] #sr
        data[3][i] += caminho[i, 0] ** 2 #sx2
        data[4][i] += caminho[i, 1] ** 2 #sy2
        data[5][i] += np.sum(caminho[i] * caminho[i]) #sr2
        data[6][i] += np.sum((caminho[i] * caminho[i])) ** (1/2) #sNorma
        data[9] [i] += 1
    else:
        i += 1
    return caminho, i

In [ ]:
def fazCaminhoEvitaPrepCres2(nPassos, data):
    """Faz um caminho aleatório registando os valores em cada passo na estrutura de dados
    nPassos: Número de passos do caminho
    data: Estrutura de dados para armazenar os valores
    return: O caminho aleatório"""
    caminho = np.zeros((nPassos + 1, 2), dtype="int")  # Inicializa o caminho
    visitados = {(0, 0)}
    data[9][0] += 1  # A posição inicial é efectuada todos os caminhos
    # Cria o caminho aleatório e guarda os dados em cada passo
    direccao = (
        0  # Direccção do último passo, 0 primeiro passo, 1 horizontal, -1 vertical
    )
    maxTentativas = 20  # N máximo de tentativas para encontrar um passo que não leve a uma posição já visitada
    for i in range(1, nPassos + 1):

        tentativas = 0  # n de tentativas efectuadas
        while tentativas < maxTentativas:
            tentativas += 1
            passo, direccaoTmp = passoPerpendicular(direccao)
            posicaoTmp = caminho[i - 1] + passo
            tmpTuple = (posicaoTmp[0], posicaoTmp[1])
            if tmpTuple not in visitados:
                break
        else:
            break  # Se o while tiver terminado por exceder o n de tentativas terminar o caminho

        caminho[i] = posicaoTmp
        direccao = direccaoTmp
        visitados.add(tmpTuple)

        data[0][i] += caminho[i, 0]  # sx
        data[1][i] += caminho[i, 1]  # sy
        data[2][i] += caminho[i]  # sr
        data[3][i] += caminho[i, 0] ** 2  # sx2
        data[4][i] += caminho[i, 1] ** 2  # sy2
        data[5][i] += np.sum(caminho[i] * caminho[i])  # sr2
        data[6][i] += np.sum((caminho[i] * caminho[i])) ** (1 / 2)  # sNorma
        data[9][i] += 1
    else:
        i += 1
    return caminho, i

In [ ]:
def amostragemCaminhosPrep(nPassos, nAmostragens, grafico = True):
    '''Cria nAmostragens de caminhos aleatórios com nPassos
    nPassos: Número de passos nos caminhos
    nAmostragens: Número de caminhos a produzir
    grafico: Se True faz o gráfico do último caminho.
    return: Os dados cumulativos das várias amostragens por passo e posições finais'''
    
    data = inicia(nPassos, nAmostragens) #Inicializa as estruturas de dados para guardar resultados
    
    for i in range(nAmostragens):
        
        caminho, n = fazCaminhoEvitaPrepCres2(nPassos, data) #Produz um caminho
        
        data[7][i] = caminho[n - 1] #Guarda a última posição do caminho
        data[8][i] = n #Guarda o número de passos do caminho efectuado
    
    maxN = int(data[8].max())
    
    data[0][:maxN] = data[0][:maxN] / data[9][ :maxN]
    data[1][:maxN] = data[1][:maxN] / data[9][ :maxN]
    data[3][:maxN] = data[3][:maxN] / data[9][ :maxN]
    data[4][:maxN] = data[4][:maxN] / data[9][ :maxN]
    data[5][:maxN] = data[5][:maxN] / data[9][ :maxN]
    data[6][:maxN] = data[6][:maxN] / data[9][ :maxN]
    
    if grafico:
        fig, ax = plt.subplots(figsize = (8, 8))
        ax.plot(caminho[:n, 0], caminho[:n, 1])
        
    return data

In [ ]:
data = amostragemCaminhosPrep(100, 10000)

In [ ]:
fig1 = xyGraficos(data)
fig1

In [ ]:
fig2 = distanciaGraficos(data)
fig2

In [ ]:
fig3 = passosGraficos(data)
fig3

In [ ]:
passosEstatistica(data)

In [ ]:
data = inicia(100, 100)

In [ ]:
%timeit -n 10000 -r 10 fazCaminhoEvitaPrepCres(100, data)

In [ ]:
data = inicia(100, 100)

In [ ]:
%timeit -n 1000 -r 10 fazCaminhoEvitaPrepCres2(100, data)

In [ ]:
data = inicia(100, 100)

In [ ]:
%timeit -n 1000 -r 10 fazCaminhoEvitaPrepCres(100, data)

In [ ]:
data = inicia(10000, 100)

In [ ]:
%timeit -n 1000 -r 10 fazCaminhoEvitaPrepCres2(1000, data)

In [ ]:
data = inicia(10000, 100)

In [ ]:
%timeit -n 1000 -r 10 fazCaminhoEvitaPrepCres(1000, data)

In [ ]:
%prun amostragemCaminhosPrep(100, 10000, False)

In [ ]:
data = inicia(100, 100)

In [ ]:
%timeit -n 1000 -r 10 fazCaminhoEvitaPrepCres2(100, data)

In [ ]:
data = inicia(10000, 100)

In [ ]:
%timeit -n 1000 -r 10 fazCaminhoEvitaPrepCres(1000, data)

In [ ]:
data = inicia(10000, 100)

In [ ]:
%timeit -n 1000 -r 10 fazCaminhoEvitaPrepCres2(1000, data)